# Simualting the waveguide mode using meep

Zhaohua Tian
2021 07 10

## Introduction
In this notebook, I will try two different ways to simualte the waveguide mode and its mode profiles and waveg vectors etc.

## The MPB Method

In [ ]:
import meep as mp
from meep import mpb
import numpy as np
from matplotlib import pyplot as plt

# Then the wavelength
omega = 1/0.78
material_SiN=mp.Medium(index=2.02)
width_guide=0.65
height_guide=0.25
resolution=200
size_cal_y=2
size_cal_z=2

geometry = [mp.Block(material=material_SiN, 
                     size=mp.Vector3(mp.inf,width_guide, height_guide),
                     center=mp.Vector3(0, 0, height_guide/2))]
geometry_lattice = mp.Lattice(size=mp.Vector3(0, size_cal_y, size_cal_z))

num_modes = 4

ms = mpb.ModeSolver(
    geometry_lattice = geometry_lattice,
    geometry         = geometry,
    resolution       = resolution,
    num_bands        = num_modes
)

E = []
store_fields = lambda ms_temp, mode: E.append(ms_temp.get_efield(which_band=mode,bloch_phase=False))    

k = ms.find_k(
    mp.NO_PARITY,
    omega, # omega
    1, # band_min
    num_modes, # band_max
    mp.Vector3(1,0,0), # korig_and_kdir
    1e-4, # tol
    omega * 2, # kmag_guess
    omega * 0.1, # kmag_min
    omega * 2, # kmag_max
    store_fields, # band_funcs
)
neff=k[0]/omega
eps = ms.get_epsilon()
eps_arr=np.transpose(np.array(eps))

In [ ]:
# %%
x_list=np.linspace(-size_cal_y/2,size_cal_y/2,resolution*2)
y_list=np.linspace(-size_cal_z/2,size_cal_z/2,resolution*2)
x_grid,y_grid=np.meshgrid(x_list,y_list)

plt.figure(figsize=(12,4))
# Plot the E fields
for mode in range(num_modes):
    Ex=np.squeeze(E[mode][:,:,0,1]).transpose()
    print('Current band: {}'.format(mode+1))
    plt.subplot(1,num_modes,1+mode)
    plt.pcolormesh(x_grid,y_grid,eps.transpose(), cmap='binary')
    plt.pcolormesh(x_grid,y_grid,np.abs(Ex), cmap='jet', alpha=0.9)
    plt.axis('off')
    st_title='Ex of mode '+str(mode+1)
    plt.title(st_title)
    plt.tight_layout()
    plt.savefig('SimulatedField.png')

and the effective index is as follows

In [ ]:
neff=np.array(k)/omega
print(neff)

## The meep method
we can also simualte the mode profile in meep by creating a simulation project. The geometery can be the same

In [ ]:
import meep as mp
from meep import mpb
import numpy as np
from matplotlib import pyplot as plt

# Then the wavelength
omega = 1/0.78
material_SiN=mp.Medium(index=2.02)
width_guide=0.65
height_guide=0.25
resolution=200
size_cal_y=2
size_cal_z=2

geometry = [mp.Block(material=material_SiN, 
                     size=mp.Vector3(mp.inf,width_guide, height_guide),
                     center=mp.Vector3(0, 0, height_guide/2))]
geometry_lattice = mp.Lattice(size=mp.Vector3(0, size_cal_y, size_cal_z))

num_modes = 4

dpml = 1
pml_layers = [mp.PML(dpml)]
sim = mp.Simulation(cell_size=mp.Vector3(0,size_cal_y,size_cal_z),
                    resolution=resolution,
                    geometry=geometry,
                    boundary_layers=pml_layers,)

sim.init_sim()
where = mp.Volume(center=mp.Vector3(), size=mp.Vector3(0,2,2))
direction = mp.X
band_num = 4
kpoint = mp.Vector3(1,0,0)
ed = sim.get_eigenmode(omega, direction, where, band_num, kpoint, eig_vol=None, match_frequency=True, 
parity=mp.NO_PARITY, resolution=resolution, eigensolver_tol=1e-12)

Be careful that we need extract the field properties via some methods.

In [ ]:
print(ed.amplitude(mp.vector3(),1))